In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import argparse

parser = argparse.ArgumentParser(description='This is a demonstration program')

device='cuda' if torch.cuda.is_available else 'gpu'

print(device)
block_size=64
batch_size=128
learning_rate=3e-4
max_iters=3000
eval_iters=100
eval_interval=500
n_embd=384
n_layer=4
n_head=4
dropout=0.2

In [9]:
chars=""
with open('ayurveda.txt','r',encoding='utf-8')as f:
    text=f.read()
    chars=sorted(set(text))#here we are taking all the letters in the text
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¼', '½', 'è', 'é', 'ñ', 'ɹ', 'ɻ', 'ɽ', 'ʀ', 'ʃ', '–', '—', '‘', '’', '“', '”', '•', '…', '⅓', '⅛']


In [10]:
#Tokenizer
string_to_int={ ch:i for i,ch in enumerate(chars)}
int_to_string={ i:ch for i,ch in enumerate(chars)}
encode=lambda s:[string_to_int[c]for c in s] #here we are making a lambda function to encode the text relative to the position in the dict
decode=lambda l:''.join([int_to_string[i]for i in l])

encoded_word=encode('!')
print(encoded_word)
decoded_word=decode(encoded_word)

data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

[2]
tensor([33, 64, 70, 68, 65, 54, 71, 53, 70, 59, 65, 64,  0, 44, 58, 55,  1, 38,
        55, 55, 54,  1, 56, 65, 68,  1, 32, 55, 51, 62, 59, 64, 57,  0, 25, 75,
        71, 68, 72, 55, 54, 51,  1, 59, 69,  1, 70, 58, 55,  1, 51, 68, 70,  1,
        65, 56,  1, 54, 51, 59, 62, 75,  1, 62, 59, 72, 59, 64, 57,  1, 59, 64,
         1, 58, 51, 68, 63, 65, 64, 75,  1, 73, 59, 70, 58,  1, 70, 58, 55,  1,
        62, 51, 73, 69,  1, 65, 56,  1, 64, 51])


In [11]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size]for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y

x,y=get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[64,  1, 65,  ...,  0, 44, 58],
        [ 1, 57, 59,  ..., 56, 65, 68],
        [70, 70, 68,  ..., 70,  1, 70],
        ...,
        [64,  1, 69,  ..., 55, 64, 64],
        [55,  1, 68,  ..., 30, 39, 39],
        [31, 42, 29,  ..., 70,  1, 66]], device='cuda:0')
targets:
tensor([[ 1, 65, 56,  ..., 44, 58, 55],
        [57, 59, 72,  ..., 65, 68, 55],
        [70, 68, 51,  ...,  1, 70, 58],
        ...,
        [ 1, 69, 71,  ..., 64, 64, 59],
        [ 1, 68, 55,  ..., 39, 39, 28],
        [42, 29, 29,  ...,  1, 66, 68]], device='cuda:0')


In [12]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print('when input is',context,'target is',target)

when input is tensor([33]) target is tensor(64)
when input is tensor([33, 64]) target is tensor(70)
when input is tensor([33, 64, 70]) target is tensor(68)
when input is tensor([33, 64, 70, 68]) target is tensor(65)
when input is tensor([33, 64, 70, 68, 65]) target is tensor(54)
when input is tensor([33, 64, 70, 68, 65, 54]) target is tensor(71)
when input is tensor([33, 64, 70, 68, 65, 54, 71]) target is tensor(53)
when input is tensor([33, 64, 70, 68, 65, 54, 71, 53]) target is tensor(70)
when input is tensor([33, 64, 70, 68, 65, 54, 71, 53, 70]) target is tensor(59)
when input is tensor([33, 64, 70, 68, 65, 54, 71, 53, 70, 59]) target is tensor(65)
when input is tensor([33, 64, 70, 68, 65, 54, 71, 53, 70, 59, 65]) target is tensor(64)
when input is tensor([33, 64, 70, 68, 65, 54, 71, 53, 70, 59, 65, 64]) target is tensor(0)
when input is tensor([33, 64, 70, 68, 65, 54, 71, 53, 70, 59, 65, 64,  0]) target is tensor(44)
when input is tensor([33, 64, 70, 68, 65, 54, 71, 53, 70, 59, 65,

In [13]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
            out[split]=losses.mean()
    model.train()
    return out

In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

In [ ]:
optimizer= torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    if iter%eval_iters==0:
        losses=estimate_loss()
        print(f"step:{iter},train loss{losses['train']},val loss:{losses['val']}")
    xb,yb=get_batch('train')
    logits,loss=model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-02.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

step:0,train loss4.633841514587402,val loss:4.6342668533325195
step:100,train loss2.4568536281585693,val loss:2.4699089527130127
step:200,train loss2.0974817276000977,val loss:2.1241822242736816
step:300,train loss1.8939992189407349,val loss:1.9348715543746948
step:400,train loss1.7530264854431152,val loss:1.8084006309509277
step:500,train loss1.6484320163726807,val loss:1.7136359214782715
step:600,train loss1.5670993328094482,val loss:1.6440151929855347
step:700,train loss1.5046941041946411,val loss:1.601082444190979
step:800,train loss1.449192762374878,val loss:1.5473490953445435
step:900,train loss1.4200825691223145,val loss:1.5183300971984863
step:1000,train loss1.3702034950256348,val loss:1.4817379713058472
step:1100,train loss1.343461036682129,val loss:1.45979642868042
step:1200,train loss1.319928765296936,val loss:1.4417338371276855
step:1300,train loss1.2966830730438232,val loss:1.4285136461257935
step:1400,train loss1.2792152166366577,val loss:1.417414903640747
step:1500,train

In [47]:
prompt = 'doshas'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

doshas after eating. If that is, try, cunter ced,
cheeses, grouly grown on the skin. Thus, job it provides
